### 1. 데이터 전처리
#### 1-1. 데이터 load 
- https://github.com/bab2min/corpus/tree/master/sentiment 에서 다운로드한 데이터 다운로드 후 load
- columns 이름 추가
- label column 생성


In [13]:
import pandas as pd

In [14]:
df = pd.read_csv("../../data/naver_shopping.txt", sep = '\t', encoding = 'utf-8', header = None)

In [15]:
df.head()

,0,1
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [16]:
df.isnull().sum()

0    0
1    0
dtype: int64

In [17]:
df.columns = ['rating', 'Review']

In [18]:
df.head()

,rating,Review
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [19]:
def rating_to_label(rating):
    if rating >=4 :
        return 1
    else:
        return 0

df['y'] = df['rating'].apply(lambda x: rating_to_label(x))

#### 1-2. 유의어 일원화
- 자체적으로 만든 유의어 사전 로딩(txt 파일)
- 비슷한 단어들을 하나의 단어로 통일하는 전처리 적용

In [20]:
f = open("../../data/review_synonym_final.txt", 'r', encoding= 'utf-8')

In [21]:
lines = f.readlines()

for idx, l in enumerate(lines):
    lines[idx] = l.strip()

In [22]:
lines_dic = {}

for idx, l in enumerate(lines):
    temp = l.split(',')
    lines_dic[temp[0]] = temp[1:]

In [23]:
for key, wordList in lines_dic.items():
    for word in wordList:
        df['Review'] = df['Review'].str.replace(word, key)

#### 1-3. 불용어 사전 편집
- "https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt" 활용
- EDA 과정에서 평점과 상관없이 빈출된 단어(배송, 구매, 생각, 사용 등) 추가

In [25]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords.append('배송')
stopwords.append('구매')
stopwords.append('생각')
stopwords.append('사용')
stopwords.append('가격')
stopwords.append('제품')

#### 1-4. 토큰화된 리뷰 컬럼 추가
- 명사, 동사, 형용사에 해당하는 2글자 이상의 형태소만 활용하는 토큰화함수 1을 적용한 리뷰 컬럼 생성
- 모든 형태소와 이모티콘을 활용하는 토큰화함수 2를 적용한 리뷰 컬럼 생성
- 향후 countervectorization, TFIDF vectorization에 사용하는 토큰화함수는 띄어쓰기 기준으로 토큰화하는 간소화된 함수 사용

In [26]:
from konlpy.tag import Okt

def tokenizer_1(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]')
    result = hangul.sub('', text)
    okt = Okt()
    Okt_morphs = okt.pos(result)
    words = []

    for word, pos in Okt_morphs:
        if pos == 'Adjective' or pos == 'Verb' or pos == 'Noun':
            if len(word) > 1 and word not in stopwords:
                words.append(word)

    return ' '.join(words)

In [31]:
okt2 = Okt()

def tokenizer_2(text):
    tokens_ko = okt2.morphs(text)

    result = []
    for word in tokens_ko:
        if word not in stopwords:
            result.append(word)
    return ' '.join(result)

In [29]:
import time
from tqdm import tqdm
import re

tokenized_1 = []

for idx, row in tqdm(df.iterrows()):
    tokenized_review = tokenizer_1(df.loc[idx, 'Review'])
    tokenized_1.append(tokenized_review)

df['tokenized_1'] = tokenized_1

200000it [7:35:35,  7.32it/s] 


In [32]:
tokenized_2 = []

for idx, row in tqdm(df.iterrows()):
    tokenized_review = tokenizer_2(df.loc[idx, 'Review'])
    tokenized_2.append(tokenized_review)

df['tokenized_2'] = tokenized_2

200000it [07:14, 460.64it/s]


In [33]:
df.head()

,rating,Review,y,tokenized_1,tokenized_2
0,5,배공빠르고 굿,1,배공 빠르고,배공 빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0,택배 엉망 놔두고가고,택배 가 엉망 이네 용 저희 집 밑 에 층 에 말 도 없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 박음질이 조금 ...,1,아주 좋아요 바지 정말 좋아서 했어요 대박 입니다 박음질 조금 어설프다하긴 편하고 ...,아주 좋아요 바지 정말 좋아서 2 개 더 했어요 이 에 대박 입니다 . 박음질 이 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0,선물 받아서 전달 했어야 하는 상품 이었는데 머그컵 와서 당황 했습니다 전화했더니 ...,선물 용 으로 빨리 받아서 전달 했어야 하는 상품 이었는데 머그컵 만 와서 당황 했...
4,5,민트색상상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1,민트 색상 예뻐요 손잡이 도로 되네요,민트 색상 상 예뻐요 . 옆 손잡이 는 거 는 용 도로 도 되네요 ㅎㅎ


In [34]:
df.to_csv("../../data/naver_shopping_preprocessed_final.csv", sep = ",", encoding = "utf-8")

In [35]:
print(df.isnull().sum())
df.info()

rating         0
Review         0
y              0
tokenized_1    0
tokenized_2    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   rating       200000 non-null  int64 
 1   Review       200000 non-null  object
 2   y            200000 non-null  int64 
 3   tokenized_1  200000 non-null  object
 4   tokenized_2  200000 non-null  object
dtypes: int64(2), object(3)
memory usage: 7.6+ MB


In [36]:
def tokenizer(text):
    return text.split()

### 2. 모델링
#### 2-1. DTM + GridSearchCV + tokenizer_1
* countervectorizer
* GridSearchCV
* Tokenizer1

In [37]:
from sklearn.model_selection import train_test_split
y = df['y']
X_train_texts, X_test_texts, y_train, y_test = train_test_split(df['tokenized_1'], y, test_size = 0.2, random_state = 0)

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

vect1 = CountVectorizer(min_df = 3, ngram_range=(1,2), tokenizer = tokenizer)
X_train_vectorized1 = vect1.fit_transform(X_train_texts)
X_test_vectorized1 = vect1.transform(X_test_texts)

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [39]:
vocablist_DTM_tk1 = [word for word, number in sorted(vect1.vocabulary_.items(), key = lambda x:x[1])]

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression(random_state = 0, solver = 'liblinear', max_iter = 1000)
params = { 'C': [0.6, 0.8, 1 , 3.5, 4.5 ] }
grid_cv = GridSearchCV(lr , param_grid=params , cv=3 ,scoring='accuracy', verbose=2 )
grid_cv.fit(X_train_vectorized1 , y_train)
print(grid_cv.best_params_ , round(grid_cv.best_score_,3))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] END ..............................................C=0.6; total time=   1.6s
[CV] END ..............................................C=0.6; total time=   1.5s
[CV] END ..............................................C=0.6; total time=   1.3s
[CV] END ..............................................C=0.8; total time=   1.8s
[CV] END ..............................................C=0.8; total time=   1.8s
[CV] END ..............................................C=0.8; total time=   1.7s
[CV] END ................................................C=1; total time=   1.9s
[CV] END ................................................C=1; total time=   2.3s
[CV] END ................................................C=1; total time=   1.7s
[CV] END ..............................................C=3.5; total time=   2.2s
[CV] END ..............................................C=3.5; total time=   5.7s
[CV] END ........................................

In [41]:
best_estimator_DTM_tk1 = grid_cv.best_estimator_
y_pred = best_estimator_DTM_tk1.predict(X_test_vectorized1)

In [42]:
print("accuracy: %.3f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.898
Precision : 0.902
Recall : 0.893
F1 : 0.897


In [43]:
import pickle

with open('saved_CounterVectorizer_tk1_final.pickle','wb') as fw:
    pickle.dump(vect1, fw)

with open('DTM_tokenizer1_gridsearch_done_final.pickle','wb') as fw:
    pickle.dump(best_estimator_DTM_tk1, fw)

In [44]:
coefficients = best_estimator_DTM_tk1.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# coefficients(계수)가 큰 값부터 내림차순으로 정렬

print('긍정적인 단어 Top 20 (높은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[:20]:
  print('{0:}({1:.3f})'.format(vocablist_DTM_tk1[word_num], coef))

긍정적인 단어 Top 20 (높은 평점과 상관관계가 강한 단어들)
맛있어요(2.721)
만족해요(2.508)
최고(2.469)
튼튼합니다(2.442)
맛있고(2.430)
예뻐요(2.386)
좋아요(2.374)
잘쓰고있어요(2.220)
편해요(2.216)
튼튼해요(2.167)
빨랐고(2.159)
빠르고(2.099)
튼튼하네요(2.093)
편리합니다(2.092)
좋았습니다(2.090)
에드(2.076)
딱좋아요(2.066)
귀엽다(2.057)
이뻐용(2.048)
만족합니다(2.037)


In [45]:
print('\n부정적인 단어 Top 20 (낮은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[-20:][::-1]: 
  print('{0:}({1:.3f})'.format(vocablist_DTM_tk1[word_num], coef))


부정적인 단어 Top 20 (낮은 평점과 상관관계가 강한 단어들)
최악(-3.543)
실망(-3.013)
비추(-2.927)
실망했어요(-2.735)
맛있어요 맛있어요(-2.677)
안좋아요(-2.635)
약해요(-2.544)
좋아요 좋아요(-2.530)
맛없어요(-2.495)
다시는(-2.474)
별로(-2.465)
떨어집니다(-2.406)
엉망(-2.364)
느려요(-2.331)
불편해요(-2.312)
다신(-2.277)
떨어져요(-2.273)
그래요(-2.254)
심하네요(-2.247)
불편합니다(-2.239)


#### 2-2. TFIDF + GridSearchCV + tokenizer_1
* TFIDF
* GridSearchCV
* Tokenizer1

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer1 = TfidfVectorizer(ngram_range=(1,2), min_df = 3, max_df = 0.9, tokenizer = tokenizer)
tfidf_vectorizer1.fit(X_train_texts)
tfidf_matrix_train = tfidf_vectorizer1.transform(X_train_texts)
tfidf_matrix_test = tfidf_vectorizer1.transform(X_test_texts)

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [47]:
vocablist_TFIDF_tk1 = [word for word, number in sorted(tfidf_vectorizer1.vocabulary_.items(), key = lambda x:x[1])]

In [48]:
lr = LogisticRegression(random_state = 0, solver = 'liblinear', max_iter = 1000)
params = { 'C': [0.6, 0.8, 1 ,3.5, 4.5 ] }
grid_cv = GridSearchCV(lr , param_grid=params , cv=3 ,scoring='accuracy', verbose=2 )
grid_cv.fit(tfidf_matrix_train , y_train)
print(grid_cv.best_params_ , round(grid_cv.best_score_,3))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] END ..............................................C=0.6; total time=   0.4s
[CV] END ..............................................C=0.6; total time=   0.3s
[CV] END ..............................................C=0.6; total time=   0.3s
[CV] END ..............................................C=0.8; total time=   0.3s
[CV] END ..............................................C=0.8; total time=   0.3s
[CV] END ..............................................C=0.8; total time=   0.3s
[CV] END ................................................C=1; total time=   0.4s
[CV] END ................................................C=1; total time=   0.4s
[CV] END ................................................C=1; total time=   0.4s
[CV] END ..............................................C=3.5; total time=   0.5s
[CV] END ..............................................C=3.5; total time=   0.7s
[CV] END ........................................

In [49]:
best_estimator_TFIDF_tk1 = grid_cv.best_estimator_
y_pred = best_estimator_TFIDF_tk1.predict(tfidf_matrix_test)

In [50]:
print("accuracy: %.3f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.899
Precision : 0.902
Recall : 0.894
F1 : 0.898


In [51]:
with open('saved_TfidfVectorizer_tk1_final.pickle','wb') as fw:
    pickle.dump(tfidf_vectorizer1, fw)

with open('TFIDF_tokenizer1_gridsearch_done_final.pickle','wb') as fw:
    pickle.dump(best_estimator_TFIDF_tk1, fw)

In [52]:
coefficients = best_estimator_TFIDF_tk1.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# coefficients(계수)가 큰 값부터 내림차순으로 정렬

print('긍정적인 단어 Top 20 (높은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[:20]:
  print('{0:}({1:.3f})'.format(vocablist_TFIDF_tk1[word_num], coef))

긍정적인 단어 Top 20 (높은 평점과 상관관계가 강한 단어들)
좋아요(10.702)
맛있어요(9.361)
예뻐요(9.091)
최고(8.217)
만족해요(8.067)
빠르고(7.594)
튼튼하고(6.309)
편해요(5.784)
맛있고(5.573)
빨랐고(5.438)
편하고(5.317)
믿고(5.283)
항상(5.266)
괜찮네요(5.265)
했어요(5.248)
좋았습니다(5.120)
강추(5.120)
확실히(5.064)
걱정 했는데(5.050)
만족(5.013)


In [53]:
print('\n부정적인 단어 Top 20 (낮은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[-20:][::-1]: 
  print('{0:}({1:.3f})'.format(vocablist_TFIDF_tk1[word_num], coef))


부정적인 단어 Top 20 (낮은 평점과 상관관계가 강한 단어들)
실망(-12.152)
별로(-11.179)
반품(-10.975)
최악(-10.473)
비추(-8.710)
그렇다(-7.477)
환불(-7.305)
엉망(-7.207)
불편해요(-7.169)
다시는(-6.969)
실망했어요(-6.916)
안좋아요(-6.889)
다신(-6.269)
불편합니다(-6.155)
떨어집니다(-6.103)
맛없어요(-6.013)
약해요(-5.918)
떨어져요(-5.892)
심하네요(-5.812)
아니네요(-5.805)


#### 2-3. DTM + GridSearchCV + tokenizer_2
* countervectorizer
* GridSearchCV
* Tokenizer2

In [54]:
X_train_texts, X_test_texts, y_train, y_test = train_test_split(df['tokenized_2'], y, test_size = 0.2, random_state = 0)

In [55]:
vect2 = CountVectorizer(min_df = 3, ngram_range=(1,2), tokenizer = tokenizer)
X_train_vectorized2 = vect2.fit_transform(X_train_texts)
X_test_vectorized2 = vect2.transform(X_test_texts)

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [56]:
vocablist_DTM_tk2 = [word for word, number in sorted(vect2.vocabulary_.items(), key = lambda x:x[1])]

In [57]:
lr = LogisticRegression(random_state = 0, solver = 'liblinear', max_iter = 1000)
params = { 'C': [0.6, 0.8, 1 ,3.5, 4.5 ] }
grid_cv = GridSearchCV(lr , param_grid=params , cv=3 ,scoring='accuracy', verbose=2 )
grid_cv.fit(X_train_vectorized2 , y_train)
print(grid_cv.best_params_ , round(grid_cv.best_score_,3))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] END ..............................................C=0.6; total time=   4.8s
[CV] END ..............................................C=0.6; total time=   4.5s
[CV] END ..............................................C=0.6; total time=   4.2s
[CV] END ..............................................C=0.8; total time=   5.3s
[CV] END ..............................................C=0.8; total time=   4.8s
[CV] END ..............................................C=0.8; total time=   4.5s
[CV] END ................................................C=1; total time=   8.3s
[CV] END ................................................C=1; total time=   4.0s
[CV] END ................................................C=1; total time=   4.6s
[CV] END ..............................................C=3.5; total time=  14.7s
[CV] END ..............................................C=3.5; total time=   8.9s
[CV] END ........................................

In [58]:
best_estimator_DTM_tk2 = grid_cv.best_estimator_
y_pred = best_estimator_DTM_tk2.predict(X_test_vectorized2)

print("accuracy: %.3f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.915
Precision : 0.914
Recall : 0.915
F1 : 0.914


In [59]:
with open('saved_CounterVectorizer_tk2_final.pickle','wb') as fw:
    pickle.dump(vect2, fw)

with open('DTM_tokenizer2_gridsearch_done_final.pickle','wb') as fw:
    pickle.dump(best_estimator_DTM_tk2, fw)

In [60]:
coefficients = best_estimator_DTM_tk2.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# coefficients(계수)가 큰 값부터 내림차순으로 정렬

print('긍정적인 단어 Top 20 (높은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[:20]:
  print('{0:}({1:.3f})'.format(vocablist_DTM_tk2[word_num], coef))

긍정적인 단어 Top 20 (높은 평점과 상관관계가 강한 단어들)
맛있어요(2.501)
만족해요(2.293)
맛있고(2.273)
예뻐요(2.170)
최고(2.119)
튼튼합니다(2.079)
좋아요(2.064)
좋았습니다(2.062)
잘 왔어요(1.957)
튼튼해요(1.946)
편합니다(1.898)
잘쓰고있어요(1.898)
편해요(1.895)
촉촉하고(1.856)
이뻐용(1.840)
빠르고(1.814)
편리합니다(1.805)
딱좋아요(1.791)
괜찮네요(1.778)
튼튼하네요(1.744)


In [61]:
print('\n부정적인 단어 Top 20 (낮은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[-20:][::-1]: 
  print('{0:}({1:.3f})'.format(vocablist_DTM_tk2[word_num], coef))


부정적인 단어 Top 20 (낮은 평점과 상관관계가 강한 단어들)
최악(-3.308)
실망(-2.755)
맛있어요 맛있어요(-2.552)
비추(-2.536)
불편해요(-2.355)
좋아요 좋아요(-2.277)
떨어져요(-2.228)
실망했어요(-2.177)
엉망(-2.167)
별로(-2.090)
안좋아요(-2.073)
맛없어요(-2.052)
불만족(-2.038)
불편합니다(-2.034)
불편하네요(-2.022)
불만(-1.933)
아니네요(-1.930)
좋아요 좋(-1.915)
환불(-1.898)
심하네요(-1.868)


#### 2-4. TFIDF + GridSearchCV + tokenizer_2
* TFIDF
* GridSearchCV
* Tokenizer2

In [62]:
tfidf_vectorizer2 = TfidfVectorizer(ngram_range=(1,2), min_df = 3, max_df = 0.9, tokenizer = tokenizer)
tfidf_vectorizer2.fit(X_train_texts)
tfidf_matrix_train = tfidf_vectorizer2.transform(X_train_texts)
tfidf_matrix_test = tfidf_vectorizer2.transform(X_test_texts)

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [63]:
vocablist_TFIDF_tk2 = [word for word, number in sorted(vect2.vocabulary_.items(), key = lambda x:x[1])]

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression(random_state = 0, solver = 'liblinear', max_iter = 1000)
params = { 'C': [0.6, 0.8, 1 ,3.5, 4.5 ] }
grid_cv = GridSearchCV(lr , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(tfidf_matrix_train, y_train)
print(grid_cv.best_params_ , round(grid_cv.best_score_,3))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.916


In [65]:
best_estimator_TFIDF_tk2 = grid_cv.best_estimator_
y_pred = best_estimator_TFIDF_tk2.predict(tfidf_matrix_test)
print("accuracy: %.3f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.915
Precision : 0.918
Recall : 0.911
F1 : 0.915


In [66]:
with open('saved_TfidfVectorizer_tk2_final.pickle','wb') as fw:
    pickle.dump(tfidf_vectorizer2, fw)

with open('TFIDF_tokenizer2_gridsearch_done_final.pickle','wb') as fw:
    pickle.dump(best_estimator_TFIDF_tk2, fw)

In [67]:
coefficients = best_estimator_TFIDF_tk2.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# coefficients(계수)가 큰 값부터 내림차순으로 정렬

print('긍정적인 단어 Top 20 (높은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[:20]:
  print('{0:}({1:.3f})'.format(vocablist_TFIDF_tk2[word_num], coef))

긍정적인 단어 Top 20 (높은 평점과 상관관계가 강한 단어들)
좋아요(14.766)
맛있어요(11.234)
예뻐요(10.507)
만족해요(9.901)
최고(9.129)
^^(8.102)
!(7.214)
잘(7.093)
빠르고(7.019)
ㅎㅎ(6.838)
편해요(6.291)
만족(6.256)
확실히(6.207)
좋았습니다(6.178)
했어요(6.100)
믿고(6.079)
튼튼하고(5.904)
괜찮네요(5.872)
좋은(5.868)
항상(5.816)


In [68]:
print('\n부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[-20:][::-1]: 
  print('{0:}({1:.3f})'.format(vocablist_TFIDF_tk2[word_num], coef))


부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)
실망(-12.320)
최악(-10.563)
반품(-10.202)
그렇다(-9.714)
별로(-9.107)
비추(-8.881)
불편해요(-8.360)
ㅡㅡ(-8.163)
못(-7.462)
엉망(-7.084)
빠르고 좋아요(-6.747)
떨어져요(-6.694)
환불(-6.693)
돈(-6.501)
불량(-6.429)
...(-6.365)
불편하네요(-6.158)
안좋아요(-6.156)
안(-6.149)
아니네요(-6.128)
